# Validating the Kroupa Initial Mass Function

This notebook demonstrates how to validate that Cosmos Genesis star generation follows the Kroupa (2001) Initial Mass Function.

## Theory

The Kroupa IMF is a broken power law:

$$\xi(m) \propto \begin{cases}
m^{-0.3} & 0.01 < m/M_\odot < 0.08 \\
m^{-1.3} & 0.08 < m/M_\odot < 0.5 \\
m^{-2.3} & 0.5 < m/M_\odot
\end{cases}$$

Where $\alpha = 2.3$ is the Salpeter slope for high-mass stars.

## References

- Kroupa, P. (2001). *On the variation of the initial mass function*. MNRAS, 322(2), 231-246.

In [ ]:
from cosmos_genesis import CosmosClient
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Initialize client
client = CosmosClient()
print("✓ Connected to Cosmos Genesis")

## 1. Query All Stars in Galaxy

In [ ]:
# Query all stars (or large sample)
galaxy_id = "spiral-sm-2arm-001"

df = client.query_to_dataframe(
    galaxy_id=galaxy_id,
    query="""
        SELECT
            stellar_mass_msun,
            spectral_type
        FROM star
        WHERE stellar_mass_msun > 0.01  -- Exclude edge cases
    """
)

print(f"Total stars: {len(df):,}")
print(f"Mass range: {df['stellar_mass_msun'].min():.3f} - {df['stellar_mass_msun'].max():.1f} M☉")

## 2. Compute Mass Function

In [ ]:
# Create logarithmic mass bins
log_mass = np.log10(df['stellar_mass_msun'])
bins = np.logspace(np.log10(0.01), np.log10(df['stellar_mass_msun'].max()), 40)
hist, bin_edges = np.histogram(df['stellar_mass_msun'], bins=bins)

# Compute bin centers and widths
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
bin_widths = bin_edges[1:] - bin_edges[:-1]

# Normalize by bin width (differential mass function)
dmf = hist / bin_widths

print(f"Created {len(bins)-1} logarithmic mass bins")

## 3. Plot Observed IMF

In [ ]:
plt.figure(figsize=(12, 8))

# Plot histogram
plt.subplot(2, 1, 1)
plt.hist(df['stellar_mass_msun'], bins=bins, edgecolor='black', alpha=0.7, label='Observed')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Stellar Mass (M☉)')
plt.ylabel('Number of Stars')
plt.title('Stellar Mass Distribution (Log-Log)')
plt.grid(alpha=0.3)
plt.legend()

# Plot differential mass function
plt.subplot(2, 1, 2)
plt.plot(bin_centers, dmf, 'o-', label='Observed dN/dm', alpha=0.7)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Stellar Mass (M☉)')
plt.ylabel('dN/dm (differential mass function)')
plt.title('Initial Mass Function')
plt.grid(alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

## 4. Fit Power Law to High-Mass Stars

In [ ]:
# Focus on high-mass regime (M > 0.5 M☉)
mask_high_mass = df['stellar_mass_msun'] > 0.5
high_mass_stars = df[mask_high_mass]

# Create bins for high-mass regime
bins_high = np.logspace(np.log10(0.5), np.log10(high_mass_stars['stellar_mass_msun'].max()), 20)
hist_high, bin_edges_high = np.histogram(high_mass_stars['stellar_mass_msun'], bins=bins_high)

bin_centers_high = (bin_edges_high[:-1] + bin_edges_high[1:]) / 2
bin_widths_high = bin_edges_high[1:] - bin_edges_high[:-1]
dmf_high = hist_high / bin_widths_high

# Remove zero bins
nonzero = dmf_high > 0
x_fit = bin_centers_high[nonzero]
y_fit = dmf_high[nonzero]

# Fit power law: dN/dm ∝ m^α
def power_law(m, A, alpha):
    return A * m**alpha

popt, pcov = curve_fit(power_law, x_fit, y_fit, p0=[1000, -2.3])
A_fit, alpha_fit = popt
alpha_err = np.sqrt(pcov[1, 1])

print("Fitted Power Law (M > 0.5 M☉):")
print(f"  α = {alpha_fit:.3f} ± {alpha_err:.3f}")
print(f"  Expected (Kroupa): α = -2.3")
print(f"  Deviation: {abs(alpha_fit + 2.3):.3f}")
print()

if abs(alpha_fit + 2.3) < 0.1:
    print("✓ Excellent agreement with Kroupa IMF!")
elif abs(alpha_fit + 2.3) < 0.2:
    print("✓ Good agreement with Kroupa IMF")
else:
    print("⚠ Significant deviation from Kroupa IMF")

## 5. Visualize Fit

In [ ]:
plt.figure(figsize=(10, 6))

# Observed data
plt.plot(bin_centers_high, dmf_high, 'o', label='Observed (M > 0.5 M☉)', alpha=0.7, markersize=8)

# Fitted power law
m_theory = np.logspace(np.log10(0.5), np.log10(high_mass_stars['stellar_mass_msun'].max()), 100)
dmf_theory = power_law(m_theory, A_fit, alpha_fit)
plt.plot(m_theory, dmf_theory, 'r-', linewidth=2, label=f'Fitted: α = {alpha_fit:.2f}')

# Kroupa theoretical
dmf_kroupa = power_law(m_theory, A_fit, -2.3)
plt.plot(m_theory, dmf_kroupa, 'g--', linewidth=2, label='Kroupa (2001): α = -2.3')

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Stellar Mass (M☉)', fontsize=12)
plt.ylabel('dN/dm', fontsize=12)
plt.title('High-Mass IMF vs Kroupa (2001)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Summary Statistics by Spectral Type

In [ ]:
# Count by spectral type
spectral_counts = df['spectral_type'].value_counts().sort_index()

print("Stars by Spectral Type:")
print("=" * 40)
for spec_type, count in spectral_counts.items():
    pct = 100 * count / len(df)
    print(f"  {spec_type}: {count:,} ({pct:.1f}%)")

print()
print(f"Total: {len(df):,} stars")

## Conclusion

This analysis validates that Cosmos Genesis correctly implements the Kroupa (2001) IMF for stellar mass generation.

## Next Steps

- **[03_stellar_evolution.ipynb](03_stellar_evolution.ipynb)**: Track stellar evolution over time
- **[04_exoplanet_stats.ipynb](04_exoplanet_stats.ipynb)**: Analyze exoplanet populations